In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import LongType, StringType, BooleanType, TimestampType
from delta.tables import DeltaTable

TABLE_SOURCE = "v_credit.bronze.chamados"
TABLE_CHAMADO_LOG = "v_credit.silver.tb_chamado_log"
TABLE_MOTIVO = "v_credit.silver.tb_motivo"
TABLE_CANAL = "v_credit.silver.tb_canal"

TABLE_TARGET_VALIDOS = "v_credit.silver.tb_chamado"
TABLE_TARGET_INVALIDOS = "v_credit.silver.tb_chamado_invalidos"

MOTIVO_CORRECTIONS = {
    r"Bloqueio de cart.*o": "Bloqueio de cartão",
    r"Contesta..o de fatura": "Contestação de fatura",
    r"D.vidas gerais sobre programa de pontos": "Dúvidas gerais sobre programa de pontos",
    r"Compra n.o autorizada": "Compra no autorizada", 
    r"Contrata..o de cart.o adicional": "Contratação de cartão adicional",
    r"Altera..o de dados cadastrais.*": "Alteração de dados cadastrais (vencimento da fatura, telefone, email)",
    r"Desbloqueio de cart.o": "Desbloqueio de cartão",
    r"Transfer.ncia de ag.ncia": "Transferência de agência",
    r"Renegocia..o de d.vida": "Renegociação de dívida",
    r"Problema com aplicativo": "Problema com aplicativo"
}

CANAL_CORRECTIONS = {
    r"(?i)U\.?R\.?A\.?": "URA",
    r"(?i)CHAT\s?BOT": "CHATBOT",
    r"(?i)^ATEND.*INICIAL$": "ATENDIMENTO INICIAL",
    r"(?i)^ATEND.*ESPECIALIZADO$": "ATENDIMENTO ESPECIALIZADO"
}

In [0]:
df_bronze = spark.table(TABLE_SOURCE)
df_chamado_log = spark.table(TABLE_CHAMADO_LOG)
df_motivo = spark.table(TABLE_MOTIVO)
df_canal = spark.table(TABLE_CANAL)

df_motivo_corrigido = df_bronze
for pattern, replacement in MOTIVO_CORRECTIONS.items():
    df_motivo_corrigido = df_motivo_corrigido.withColumn(
        "motivo", F.regexp_replace(F.col("motivo"), pattern, replacement)
    )

df_canal_corrigido = df_motivo_corrigido
for pattern, replacement in CANAL_CORRECTIONS.items():
    df_canal_corrigido = df_canal_corrigido.withColumn(
        "canal", F.regexp_replace(F.upper(F.trim(F.col("canal"))), pattern, replacement)
    )


In [0]:
df_with_lookups = (
    df_canal_corrigido.alias("c")
    .join(
        df_chamado_log.select("cd_chamado", "dh_abertura", "dh_inicio", "dh_fim").alias("log"),
        F.col("c.id_chamado") == F.col("log.cd_chamado"),
        "left"
    )
    .join(
        df_motivo.select("cd_motivo", "ds_motivo").alias("m"),
        F.lower(F.trim(F.col("c.motivo"))) == F.lower(F.trim(F.col("m.ds_motivo"))),
        "left"
    )
    .join(
        df_canal.select("cd_canal", "nm_canal").alias("can"),
        F.trim(F.col("c.canal")) == F.col("can.nm_canal"),
        "left"
    )
)

In [0]:
df_transformado = (
    df_with_lookups
    .select(
        F.col("c.id_chamado").cast(LongType()).alias("cd_chamado"),

        F.sha2(F.col("c.id_cliente").cast(StringType()), 256).alias("cd_cliente"),

        F.col("m.cd_motivo").cast(LongType()).alias("cd_motivo"),
        F.col("can.cd_canal").cast(LongType()).alias("cd_canal"),

        F.when(
            F.upper(F.regexp_replace(F.col("c.resolvido"), r"(?i)n.o", "NAO")) == "SIM",
            F.lit(True)
        ).when(
            F.upper(F.regexp_replace(F.col("c.resolvido"), r"(?i)n.o", "NAO")) == "NAO",
            F.lit(False)
        ).otherwise(None).cast(BooleanType()).alias("st_resolvido"),

        (F.col("log.dh_inicio").cast("long") - F.col("log.dh_abertura").cast("long")).alias("tm_espera"),
        (F.col("log.dh_fim").cast("long") - F.col("log.dh_inicio").cast("long")).alias("tm_duracao"),

        F.when(
            (F.col("c.id_atendente").isNull()) |
            (F.trim(F.col("c.id_atendente").cast(StringType())) == ""),
            F.lit(None)
        ).otherwise(F.col("c.id_atendente").cast(LongType())).alias("cd_atendente"),

        F.col("c.ingestion_timestamp").alias("dt_ingestion"),
        F.coalesce(F.col("c.origem"), F.lit("chamados")).alias("dc_origem")
    )
    .dropDuplicates(["cd_chamado"])
)


In [0]:
df_validacao = (
    df_transformado
    .withColumn("flag_pk_valida", F.col("cd_chamado").isNotNull())
    .withColumn("flag_fk_cliente_valida", F.col("cd_cliente").isNotNull())
    .withColumn("flag_fk_motivo_valida", F.col("cd_motivo").isNotNull())
    .withColumn("flag_fk_canal_valida", F.col("cd_canal").isNotNull())
    .withColumn("flag_qualidade",
        F.when(
            F.col("flag_pk_valida") &
            F.col("flag_fk_cliente_valida") &
            F.col("flag_fk_motivo_valida") &
            F.col("flag_fk_canal_valida"),
            F.lit("OK")
        ).otherwise(F.lit("ERRO"))
    )
)

df_silver = (
    df_validacao
    .filter(F.col("flag_qualidade") == "OK")
    .select(
        "cd_chamado", "cd_cliente", "cd_motivo", "cd_canal", "st_resolvido",
        "tm_espera", "tm_duracao", "cd_atendente", "dt_ingestion", "dc_origem"
    )
)

df_invalidos = df_validacao.filter(F.col("flag_qualidade") == "ERRO")

total_validos = df_silver.count()
total_invalidos = df_invalidos.count()

print(f"✅ Transformação e validação concluídas: {total_validos} válidos | {total_invalidos} inválidos")

In [0]:
delta_table = DeltaTable.forName(spark, TABLE_TARGET_VALIDOS)

delta_table.alias("destino").merge(
    df_silver.alias("origem"),
    "destino.cd_chamado = origem.cd_chamado"
).whenMatchedUpdateAll(
).whenNotMatchedInsertAll(
).execute()

print(f"✅ Tabela {TABLE_TARGET_VALIDOS} atualizada com sucesso!")

df_invalidos.write.format("delta").mode("overwrite").saveAsTable(TABLE_TARGET_INVALIDOS)
print(f"✅ Tabela {TABLE_TARGET_INVALIDOS} atualizada para auditoria!")